## Principal Coordinates Plots

Before running this notebook:<br><br>
<li><a href="https://docs.qiime2.org/2018.11/install/native/#install-qiime-2-within-a-conda-environment"> install Qiime2</a></li>
<li> activate the Qiime2 conda environment by typing:</li>

`source activate qiime2-2019.4` <br><br>


<li> open this jupyter notebook from within your Qiime2 conda environment</li>

In [1]:
import pandas as pd
import qiime2 as q2
import os
path = '/home/people/julcor/.conda/envs/qiime2-2019.4' # define path to your qiime2 conda environment
os.environ['PATH'] += ':'+path

## Perform a Principal Coordinates Analysis (PCoA) and visualize results interactively in Emperor

load feature table

In [9]:
ft = pd.read_csv('PPG_KMF_resultingdatamatrix_GNPS.csv', sep = ';', index_col= 0)
#ft = pd.read_csv('PPG_KMF_resultingdatamatrix_GNPS_withoutOutliers_withPools.csv', sep = ';', index_col= 0) 
#ft = pd.read_csv('PPG_KMF_resultingdatamatrix_GNPS_withoutOutliers.csv', sep = ';', index_col= 0)

In [11]:
ft.columns = ft.columns.str.replace(' filtered Peak area','')

load metadata 

In [13]:
md = pd.read_csv('metadata.txt', sep = '\t') 

In [15]:
md = md.rename(columns={'filename': '#SampleID'})

are there any samples missing in the metadata?

In [17]:
set(ft.columns) - set(md['#SampleID']) 

{'row m/z', 'row retention time'}

In [18]:
ft = ft.drop(['row m/z','row retention time'], axis=1)

write formatted metadata and feature table to file

In [19]:
md.to_csv("Metadata_Qiime2.tsv",sep='\t',index=False)

In [20]:
ft.to_csv("Featuretable_Qiime2.tsv",sep='\t',index=True)

convert feature table to the Qiime2 .qza format

In [21]:
! biom convert -i Featuretable_Qiime2.tsv -o FeatureTable_MultStats.biom --table-type="OTU table" --to-hdf5

In [ ]:
! qiime tools import --type 'FeatureTable[Frequency]' --input-path FeatureTable_MultStats.biom --output-path FeatureTable_MultStats.qza

#### Calculate Bray-Curtis distances and visualize PCoA in Emperor

In [16]:
! mkdir MultStats

In [17]:
! qiime diversity beta --i-table FeatureTable_MultStats.qza --p-metric braycurtis --o-distance-matrix MultStats/bray_distance_matrix.qza
! qiime diversity pcoa --i-distance-matrix MultStats/bray_distance_matrix.qza --o-pcoa MultStats/bray_distance_matrix_PcoA.qza
! qiime emperor plot --i-pcoa MultStats/bray_distance_matrix_PcoA.qza --m-metadata-file Metadata_Qiime2.tsv --o-visualization MultStats/bray_distance_matrix_PcoA.qzv

Saved DistanceMatrix to: MultStats/bray_distance_matrix.qza
Saved PCoAResults to: MultStats/bray_distance_matrix_PcoA.qza
Saved Visualization to: MultStats/bray_distance_matrix_PcoA.qzv


#### Calculate Canberra distances and visualize PCoA in Emperor

In [18]:
! qiime diversity beta --i-table FeatureTable_MultStats.qza --p-metric canberra --o-distance-matrix MultStats/canberra_distance_matrix.qza
! qiime diversity pcoa --i-distance-matrix MultStats/canberra_distance_matrix.qza --o-pcoa MultStats/canberra_distance_matrix_PcoA.qza
! qiime emperor plot --i-pcoa MultStats/canberra_distance_matrix_PcoA.qza --m-metadata-file Metadata_Qiime2.tsv --o-visualization MultStats/canberra_distance_matrix_PcoA.qzv

Saved DistanceMatrix to: MultStats/canberra_distance_matrix.qza
Saved PCoAResults to: MultStats/canberra_distance_matrix_PcoA.qza
Saved Visualization to: MultStats/canberra_distance_matrix_PcoA.qzv


#### Calculate Jaccard distances and visualize PCoA in Emperor

In [19]:
! qiime diversity beta --i-table FeatureTable_MultStats.qza --p-metric jaccard --o-distance-matrix MultStats/jaccard_distance_matrix.qza
! qiime diversity pcoa --i-distance-matrix MultStats/jaccard_distance_matrix.qza --o-pcoa MultStats/jaccard_distance_matrix_PcoA.qza
! qiime emperor plot --i-pcoa MultStats/jaccard_distance_matrix_PcoA.qza --m-metadata-file Metadata_Qiime2.tsv --o-visualization MultStats/jaccard_distance_matrix_PcoA.qzv

Saved DistanceMatrix to: MultStats/jaccard_distance_matrix.qza
Saved PCoAResults to: MultStats/jaccard_distance_matrix_PcoA.qza
Saved Visualization to: MultStats/jaccard_distance_matrix_PcoA.qzv


#### Calculate Robust Aitchison distances and visualize PCoA in Emperor

In [20]:
! qiime deicode rpca \
    --i-table FeatureTable_MultStats.qza \
    --p-min-feature-count 0 \
    --p-min-sample-count 0 \
    --o-biplot MultStats/deicode_distance_matrix_PCoA.qza \
    --o-distance-matrix MultStats/deicode_distance_matrix.qza

Saved PCoAResults % Properties('biplot') to: MultStats/deicode_distance_matrix_PCoA.qza
Saved DistanceMatrix to: MultStats/deicode_distance_matrix.qza


In [21]:
! qiime emperor biplot \
    --i-biplot MultStats/deicode_distance_matrix_PCoA.qza \
    --m-sample-metadata-file Metadata_Qiime2.tsv \
    --o-visualization MultStats/deicode_distance_matrix_PCoA.qzv \
    --p-number-of-features 8

Saved Visualization to: MultStats/deicode_distance_matrix_PCoA.qzv
